In [22]:
from sentence_transformers import SentenceTransformer, util


# paraphrase-multilingual-MiniLM-L12-v2 -> best for multilingual
# sentence-transformers/all-MiniLM-L6-v2 -> best for english [multi-qa-MiniLM-L6-cos-v1 alternative]

hardware_requirements = {
    "CPU": "Intel Core i5 or equivalent (4 cores or more)",
    "GPU": "Not required (works on CPU)",
    "RAM": "16 GB",
    "Storage": "20-50 GB SSD",
    "Network": "1Gbps connection for real-time serving",
    "Operating System": "Linux-based systems (e.g., Ubuntu, CentOS, Debian)",
    "Python Version": "Python 3.7+"
}

# Load the model (Runs on CPU)
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Example intents
intents = {
    "greeting": ["Hello!", "Hi there!", "Good morning", "Hey!", "How's it going?", "What's up?"],
    "goodbye": ["Goodbye!", "See you later!", "Take care", "Bye!", "See you soon", "Farewell"],
    "weather": ["What's the weather like?", "Tell me the forecast", "How's the weather today?", "Is it raining?", "What's the temperature?", "Do I need an umbrella today?"],
    "order_status": ["Where is my order?", "Check my order status", "Is my order shipped?", "When will my order arrive?", "Track my order", "What is the status of my order?"],
    "Annual Leave Allowance": ["How much annual leave do I have?", "What is the annual leave allowance?", "How many days of leave can I take?", "What is the policy for annual leave?"],
    "Annual Leave Carryover Rules": ["Can I carry over my annual leave?", "What happens if I don't use all my leave?", "How much leave can I carry over to the next year?", "What is the carryover policy for annual leave?"],
    "Attendance": ["How is attendance tracked?", "What is the attendance policy?", "How do I mark my attendance?", "Can I check my attendance?", "What happens if I miss work?", "How is attendance reported?"],
    "Corporate Offers": ["What are the corporate offers?", "Can I get discounts through corporate offers?", "Tell me about the corporate deals available", "Are there any special offers for employees?", "What corporate offers are available this month?"],
    "Leave Application Process": ["How do I apply for leave?", "What is the process for applying for leave?", "Can I apply for leave online?", "How do I submit a leave request?", "What steps do I follow to apply for leave?"],
    "Leave Encashment Policy": ["What is the leave encashment policy?", "Can I encash my unused leave?", "How does leave encashment work?", "What is the procedure for leave encashment?", "Is there a limit on leave encashment?", "How is leave encashment calculated?"],
    "Leave Policy": ["What is the leave policy?", "How many days of leave am I entitled to?", "What types of leave are available?", "Can I take leave for personal reasons?", "What are the rules for taking leave?", "Is there any restriction on taking leave?"]
}


# Encode intent sentences
intent_embeddings = {intent: [model.encode(sent) for sent in examples] for intent, examples in intents.items()}



def find_intent(user_query):
    query_embedding = model.encode(user_query)
    best_intent, highest_score = None, 0.0

    for intent, examples in intent_embeddings.items():
        for example_embedding in examples:
            score = util.cos_sim(query_embedding, example_embedding).item()
            if score > highest_score:
                highest_score, best_intent = score, intent

    return best_intent, highest_score

In [29]:
queries = ["কেমন আছেন আনপ্নারা", "আজকের জন্য বিদায় ","আজকের আকাশ কেমন ","Is it sunny day?","Today is too much hot","Where’s my delivery?",
           "Has my package been shipped?","How many vacation days do I have?","What happens if I miss a day of work?",
           "Can I cash out unused leave?","What happens if I need to take emergency leave?"
        ]

for query in queries:
    intent, score = find_intent(query)
    print(f'Query : {query} -> Intent : {intent} -> Score : {score}')

Query : কেমন আছেন আনপ্নারা -> Intent : greeting -> Score : 0.47236761450767517
Query : আজকের জন্য বিদায়  -> Intent : goodbye -> Score : 0.7884892225265503
Query : আজকের আকাশ কেমন  -> Intent : weather -> Score : 0.5991542935371399
Query : Is it sunny day? -> Intent : weather -> Score : 0.6514041423797607
Query : Today is too much hot -> Intent : weather -> Score : 0.5234214067459106
Query : Where’s my delivery? -> Intent : order_status -> Score : 0.650810956954956
Query : Has my package been shipped? -> Intent : order_status -> Score : 0.7269501686096191
Query : How many vacation days do I have? -> Intent : Annual Leave Allowance -> Score : 0.6641182899475098
Query : What happens if I miss a day of work? -> Intent : Attendance -> Score : 0.9123213291168213
Query : Can I cash out unused leave? -> Intent : Leave Encashment Policy -> Score : 0.8274987936019897
Query : What happens if I need to take emergency leave? -> Intent : Leave Policy -> Score : 0.6962064504623413
